In [ ]:
soap_notes_fp = "sample_output1.csv"
bhc_html_fp = "eval.csv"
scores_fp = "scores.csv"

In [ ]:
# Parameters
soap_notes_fp = "soap_note_sample1test_10415772-DS-14.csv"
bhc_html_fp = "sample1html_10415772-DS-14.html"
scores_fp = "sample1scores_10415772-DS-14.csv"


# Brief Hospital Course Generation

This notebook performs step 2 of the Brief Hospital Course pipeline, in which we generate a brief hospital course from a set of service-level SOAP notes passed into a GPT-3.5 model. 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import ast
import numpy as np

from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

import os
import openai

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

In [ ]:
from prompt_functions import create_brief_hospital_course_prompts
from openai_utils import num_tokens_from_string, completion_with_backoff

In [ ]:
import openai
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
# engine = "decile-gpt-35-turbo-16k"
engine = "decile-gpt-4-128K"


## Read in GPT-Generated SOAP Notes

In [ ]:
soap_notes = pd.read_csv(soap_notes_fp)

## Read in Prompts

In [ ]:
prompts = pd.read_csv("bhc_prompts.csv")

In [ ]:
prompts

## Read in Radiology Reports

In [ ]:
radiology = pd.read_csv("/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/radiology.csv.gz")

## Read in Encounter-Level Structured Data

In [ ]:
###### 
# Challenge Data

# discharge summaries
discharges = pd.read_csv("/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/discharge.csv.gz")

# ed stays
edstays = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/edstays.csv.gz')

# triage
triage = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/triage.csv.gz')

###### 
# MIMIC-IV Data

# ward transfers
transfers = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_transfers.pkl')

# higher-level services (ICU, CARD, etc)
services = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_services.pkl')

# get patient info
pts = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_patients.pkl')

# admission demographics
admissions = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_admissions.pkl')

In [ ]:
# diagnoses
diags = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_diagnoses_icd.pkl')

### Clean up/type cast data

In [ ]:
discharges = discharges.astype({"charttime":"datetime64[ns]",
                               "storetime":"datetime64[ns]"})

## Generate SOAP Note List Prompt

In [ ]:
soap_notes

In [ ]:
discharges[discharges['hadm_id'] == soap_notes.iloc[0]['hadm_id']].squeeze()

## Create SOAP notes from GPT API (0-shot)

In [ ]:
bhc_prompt_zeroshot = create_brief_hospital_course_prompts(discharges[discharges['hadm_id'] == soap_notes.iloc[0]['hadm_id']].squeeze(),
                                                  soap_notes, prompts, 
                                                  edstays, radiology, 
                                                  pts, triage, transfers, diags,
                                                  shots=None
                                                 )

In [ ]:
num_tokens_from_string(bhc_prompt_zeroshot, "cl100k_base")

In [ ]:
messages = [{"role":"system","content":prompts.loc[prompts['prompt_name'] == "bhc_system_message", "prompt"].squeeze()}]

gpt_bhc_prompt = {"role":"user",
                 "content":bhc_prompt_zeroshot}

messages.append(gpt_bhc_prompt)

print(f"Brief Hospital Course 1-shot Prompt: {messages}")

In [ ]:
zeroshot_completion = completion_with_backoff(engine=engine,
                        messages=messages,
                        )


In [ ]:
zero_shot_output = zeroshot_completion['choices'][0]['message']['content']; print(zero_shot_output)

## Create SOAP notes from GPT API (2-shot)

It's really one shot I'm just testing two of the same example

In [ ]:
targets = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/discharge_target.csv.gz')

In [ ]:
bhc_oneshot_sample = targets.sample(1)['brief_hospital_course'].squeeze()

In [ ]:
bhc_prompt_2shot = create_brief_hospital_course_prompts(discharges[discharges['hadm_id'] == soap_notes.iloc[0]['hadm_id']].squeeze(),
                                                  soap_notes, prompts, 
                                                  edstays, radiology, 
                                                  pts, triage, transfers, diags,
                                                  shots=[bhc_oneshot_sample, bhc_oneshot_sample]
                                                 )

In [ ]:
num_tokens_from_string(bhc_prompt_2shot, "cl100k_base")

In [ ]:
messages = [{"role":"system","content":prompts.loc[prompts['prompt_name'] == "bhc_system_message", "prompt"].squeeze()}]

gpt_bhc_prompt = {"role":"user",
                 "content":bhc_prompt_2shot}

messages.append(gpt_bhc_prompt)

print(f"Brief Hospital Course N-shot Prompt: {messages}")

In [ ]:
nshot_completion = completion_with_backoff(engine=engine,
                        messages=messages,
                        )


In [ ]:
n_shot_output = nshot_completion['choices'][0]['message']['content']; print(n_shot_output)

In [ ]:
n_shot_output

## Compute ROUGE Scores from Output

In [ ]:
from rouge_score import rouge_scorer


In [ ]:
# discharges[discharges['hadm_id'] == soap_notes.iloc[0]['hadm_id']].squeeze()
pt_target = targets[targets['hadm_id'] == soap_notes.iloc[0]['hadm_id']]['brief_hospital_course'].squeeze()

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1',  "rouge2", 'rougeL'], use_stemmer=True)
scores1 = scorer.score(zero_shot_output,
                      pt_target)

In [ ]:
scores1

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)
scores2 = scorer.score(n_shot_output,
                      pt_target)

In [ ]:
scores2

# Qualitative Eval

## Final outputs


In [ ]:
pd.DataFrame.from_records([scores1, scores2], index=["zero-shot", "2-shot"]).to_csv(scores_fp)

In [ ]:
eval = pd.Series([n_shot_output, zero_shot_output, pt_target], index=["gpt-2-shot", "gpt-zero-shot", "gold-standard"])

In [ ]:
def add_line_breaks(text):
    return text.replace(r'\n', r'<br>')

In [ ]:
output_str = eval.to_frame().to_html(escape=False).replace(r"\n","<br>")
with open(bhc_html_fp, 'w') as file:
    file.write(output_str)
    


In [ ]:
print(n_shot_output)

In [ ]:
print(zero_shot_output)

In [ ]:
print(pt_target)

## SOAP Notes

In [ ]:
print("\n----------------------------------------------------------------------\n")
print(soap_notes['gpt_SOAP_note'].tolist()[0])
print("\n----------------------------------------------------------------------\n")
print(soap_notes['gpt_SOAP_note'].tolist()[1])

## Full Discharge Summary

In [ ]:
print(discharges[discharges['hadm_id'] == soap_notes.iloc[0]['hadm_id']]['text'].squeeze())